In [146]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import time
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from pymorphy2 import MorphAnalyzer
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iraid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Скачивание описаний вакансий

## С использованием API ΗΗ

#### Формирование списка id вакансий для последующего скачивания описаний и прочих параметров каждой из них

In [405]:
page=0
per_page=100


url=f'https://api.hh.ru/vacancies?per_page={per_page}&page={page}'

res=requests.get(url)

vacancies=res.json()

vacancies_id=set({})

while 'items' in vacancies.keys():
    for i in range(len(vacancies['items'])):
        vacancies_id.add(vacancies['items'][i]['id'])
    
    page+=1
    
    url=f'https://api.hh.ru/vacancies?per_page={per_page}&page={page}'
    
    res=requests.get(url)

    vacancies=res.json()
    
    #print(url)

vacancies_id=list(vacancies_id)

In [406]:
print('Количество вакансий: {0}'.format(len(vacancies_id)))

Количество вакансий: 2000


#### Сохранение списка id вакансий

In [407]:
#f=open(r'D:\Дима\Учеба\ML\DA.SberUniversity\Final project\hh_vacancies_id.txt','w')
#for vac in vacancies_id:
    #f.write(str(vac)+'\n')
#f.close()

#### Сбрка и предобработка описаний вакансий

In [408]:
# функция, удаляющая все символы, кроме букв, цифр, пробелов (а также группы пробелов), подчеркиваний, и приводящая слова к нормальной форме
def cleaner_1(s):
    
    morph=MorphAnalyzer()
    
    s=re.sub('\W',' ',s)
    clean=[]
    for token in s.split(' '):
        if token!='':
            clean.append(morph.normal_forms(token)[0])
    return ' '.join(clean)    

cnt_vac=len(vacancies_id)


#собираем из вакансий необходимую информацию

agg_description=[]


for i in range(len(vacancies_id[:cnt_vac])):
    
    
    url_one=f'https://api.hh.ru/vacancies/{vacancies_id[i]}'

    res_one=requests.get(url_one)

    vacancy=res_one.json()

    def ValExist(d,k):
        if k in d.keys() and d[k] is not None:
            return True
        else:
            return False

    if ValExist(vacancy,'name'):
        name=vacancy['name']
    else:
        name=''

    if ValExist(vacancy,'area'):
        area=vacancy['area']['name']
    else:
        area=''

    if ValExist(vacancy,'salary'):
        rep=lambda x: 'рубль' if x=='RUR' else 'валюта'
        currency=rep(vacancy['salary']['currency'])
    else:
        currency=''

    if ValExist(vacancy,'address'):
        if ValExist(vacancy['address'],'city'):
            city = vacancy['address']['city']
        else:
            city=''
        if ValExist(vacancy['address'],'street'):
            street = vacancy['address']['street']
        else:
            street=''
        if ValExist(vacancy['address'],'building'):
            building = vacancy['address']['building']
        else:
            building=''
        
        address= city + ' ' + street + ' ' + building
    else:
        address=''

    if ValExist(vacancy,'experience'):
        experience = vacancy['experience']['name']
    else:
        experience=''

    if ValExist(vacancy,'experience'):
        schedule=vacancy['schedule']['name']
    else:
        schedule=''

    if ValExist(vacancy,'employment'):
        employment=vacancy['employment']['name']
    else:
        employment=''

    if ValExist(vacancy,'description'):
        reg=r'</li>|<li>|<strong>|</strong>|<br />|</ul>|<ul>|</p>|<p>|<hr />|<ol>|<em>|</em>'
        description=re.sub(reg,'',vacancy['description'])
    else:
        description=''

    if ValExist(vacancy,'key_skills'):
        key_skills=''
        for ks in vacancy['key_skills']:
            key_skills+=ks['name']+' '
    else:
        key_skills=''

    if ValExist(vacancy,'specializations'):
        specializations=set({})
        for v in vacancy['specializations']:
            specializations.add(v['name'])
            specializations.add(v['profarea_name'])
        specializations=' '.join(list(specializations))
    else:
        specializations=''

    if ValExist(vacancy,'professional_roles'):
        professional_roles=set({})
        for v in vacancy['professional_roles']:
            professional_roles.add(v['name'])
        professional_roles=' '.join(list(professional_roles))
    else:
        professional_roles=''

    if ValExist(vacancy,'employer'):
        employer=vacancy['employer']['name']
    else:
        employer=''

    if ValExist(vacancy,'working_time_intervals'):
        working_time_intervals=set({})
        for v in vacancy['working_time_intervals']:
            working_time_intervals.add(v['name'])
        working_time_intervals=' '.join(list(working_time_intervals))
    else:
        working_time_intervals=''

    agg_vac= name + ' ' + area + ' ' + currency + ' ' + address + ' ' + experience + ' ' + schedule + ' ' + employment \
                     + ' ' + description + ' ' + key_skills + ' ' + specializations + ' ' + professional_roles \
                     + ' ' + employer + ' ' + working_time_intervals
    
    
    agg_description.append(cleaner_1(agg_vac))

dfvac=pd.DataFrame({'id_vacancies':vacancies_id[:cnt_vac],'full_description':agg_description})

#### Примеры резюме

In [409]:
resume_1="""Образование: сентябрь 1996 г. – июнь 1998 г. Техникум учёта и механизации, финансовый факультет, специальность – «документоведение», диплом младшего специалиста (дневное отделение). 
Дополнительное образование: январь 2007 г. – апрель 2007 г. Курсы по изучению программы 1С. 

Опыт работы: 

Архивариус август 1998 г. – сентябрь 2005 г. ООО «Оптокомг», г. Киров 
Функциональные обязанности: 
— хранение архивных документов; 
— регистрация и систематизация архивных документов; 
— выдача документов и контроль их возврата; 
— периодическая инвентаризация. 

Кладовщик сентябрь 2005 г. – май 2008 г. ПАО «Спец-торг», г. Киров
Функциональные обязанности: 
— приём, хранение и отпуск товарно-материальных ценностей; 
— обеспечение сохранности ценностей на складе; 
— учёт складских операций; 
— составление отчётности. 

Кладовщик июль 2008 г. – февраль 2014 г. ПАО «Холдинг-групп», г. Киров
Функциональные обязанности: 
— ведение учёта складских операций; 
— проведение разгрузочно-погрузочной работы с продуктами питания;
— обеспечение целостности и сохранности материальных ценностей на складе; 
— обеспечение правильной и безопасной эксплуатации оборудования. 

Профессиональные навыки: 
— Уверенный пользователь ПК, знание офисных программ, в т.ч. 1С; 
— Владение офисной техникой (сканер, принтер, факс); 
— Знание норм складского учёта; 
— Знание складской логистики; 
— Владение языками: русский свободно; английский – со словарём. 

Личные качества: 
— ответственность;
— усидчивость; 
— организованность; 
— честность; 
— внимательность к мелочам; 
— хорошая память. 

Дополнительные сведения: 
Семейное положение: женат. 
Дети: есть. 
Возможность командировок: нет."""


In [410]:
resume_2="""
Образование физик, ФИЗТЕХ.
Знаю математический анализ, теорию графов, математическую статистику и теорию вероятности.
Умею строить математические модели спроса, количества продаж, структуры сети ретэйла.
Владею С++, С#, JavaScript, Python, VBA.
Призер соревнований по ML: умею строить сложные модели и их ансамбли (XGBoost,CatBoost,pipeline,TensorFlow) с реализацией в web (JavaScript)
Желаемое место работы: г. Казань. З/п 120000 рублей.
Ответственный, любопытный, перфекционист.
"""

In [411]:
resume_3="""Программист высокого уровня. C#,C++,JavaScript,Python. Бэкенд разработка."""

#### Выбираем одно из резюме и делаем его токенизацию, лемматизацию и чистку от знаков препинания

In [412]:
resume=cleaner_1(resume_3)

#### Объединяем вакансии и резюме для единой векторизации

In [413]:
train_corpus=dfvac['full_description'].values
train_corpus=np.append(train_corpus,resume)

#### Добавляем стоп-слова для исключения из мешка 

In [414]:
stop=stopwords.words('russian')

#### Проводим "обучение" для последующей векторизации вакансий и резюме

In [415]:
text_transformer=TfidfVectorizer(stop_words=stop)
text_transformer.fit(train_corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', '...гда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

#### Проводим векторизацию

In [416]:
# "векторизуем" вакансии
vectorized_vacancies=text_transformer.transform(dfvac['full_description'])

In [417]:
# "векторизуем" резюме
resume=[resume]
vectorized_resume=text_transformer.transform(resume)

#### Собираем результат в единый DataFrame

In [418]:
# формируем датафрейм с кодом вакансии и косинусным расстоянием
TOP_sutable_vac=pd.DataFrame({'id_vacancies':dfvac['id_vacancies'].values,\
                              'cosine_metric':cosine_similarity(vectorized_vacancies,vectorized_resume).T[0]})

In [419]:
# сортируем вакансии по убыванию "близости" к резюме
TOP_sutable_vac.sort_values(by='cosine_metric',ascending=False,inplace=True)

In [420]:
TOP_sutable_vac.head()

,id_vacancies,cosine_metric
693,55427361,0.264255
1692,55346662,0.244388
78,55570434,0.243958
650,55896729,0.239641
127,66131995,0.221132


In [421]:
# выбираем ТОП=10 вакансий и выводим их названия и описания
TOP=10
for i in range(TOP):
    
    url_one=f'https://api.hh.ru/vacancies/{TOP_sutable_vac.iloc[i,0]}'

    res_one=requests.get(url_one)

    vacancy=res_one.json()
    print('{0}:\n{1} \n\n'.format(vacancy['name'],re.sub(reg,'',vacancy['description'])))

Начинающий python backend разработчик:
Компания Webim предоставляет платформу для общения компаний с их клиентами. Наш сервис ежедневно обрабатывает сотни тысяч обращений Чем предстоит заниматься:   Разработка бэкенд-компонентов в большом проекте чата для организаций;   Писать код, проходить код ревью, ревьюить чужой код, давайть оценки, решать сложные проблемы, фиксить баги и т.д.   Работать с командными инструментами: git/BitBucket, JIRA, PyCharm, WebStorm, IDEA Ultimate, Confluence, Jenkins и др.   Требования/пожелания:  Python; Базовое понимание CSS\HTML\JS будет преимуществом; Angular будет преимуществом; Опыт работы с базами данных (требуется знание raw SQL, знакомство с PostgreSQL, MySQL, Clickhouse, а также Elastic Search будет плюсом); Знание алгоритмов и структур данных; Умение настраивать окружение WEB-серверов (nginx будет Вашим преимуществом); Способность принимать и вовремя давать обратную связь по поводу того, что происходит в проекте; Приветствуется участие в олимпиадах